In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision

import numpy as np
import time
from os.path import join
import copy

from loader.idrid_loader import create_idrid_loader

from torchvision import transforms
import keras_transforms
from utilities import color_transform

from utilities import metrics

from models.refinenet_4cascade import RefineNet4Cascade
#from models.unet import UNet

import matplotlib.pyplot as plt
%matplotlib inline

/home/reza/anaconda3/envs/deep1/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load Data

In [2]:
image_size = 512
mask_size = 128

data_dir = './inputs'
output_dir = './weights'

In [3]:
def train_transforms(image, mask):
    
    # Convert PIL image to 3D numpy
    image = keras_transforms.img_to_array(image, data_format='channels_last')
    mask = keras_transforms.img_to_array(mask, data_format='channels_last')    
    
    result = keras_transforms.random_horizontal_flip([image, mask])
    result = keras_transforms.random_rotation(result, 360, fill_mode='constant')
    result = keras_transforms.random_zoom(result, (1/1.15, 1.15))
    result = keras_transforms.random_shift(result, 0.08, 0.08, fill_mode='constant')
    image, mask = result[0], result[1]    
    image = color_transform.augment_color(image)
    
    # Convert 3D numpy to PIL image
    image = keras_transforms.array_to_img(image, data_format='channels_last', scale=False)
    mask = keras_transforms.array_to_img(mask, data_format='channels_last', scale=False)
    image = transforms.Resize((image_size, image_size))(image)
    mask = transforms.Resize((mask_size, mask_size))(mask)
        
    # Convert PIL image to Pytorch tensor
    image = transforms.ToTensor()(image)
    mask = transforms.ToTensor()(mask)
    
    image = transforms.Normalize([0.03305855, 0.03305855, 0.01814391], 
                         [0.1787895, 0.12863145, 0.0794853])(image)
    
    return image, mask

def valid_transforms(image, mask):
    # Convert PIL image to 3D numpy
    image = keras_transforms.img_to_array(image, data_format='channels_last')
    mask = keras_transforms.img_to_array(mask, data_format='channels_last')
    image = color_transform.augment_color(image)
    
    # Convert 3D numpy to PIL image
    image = keras_transforms.array_to_img(image, data_format='channels_last', scale=False)
    mask = keras_transforms.array_to_img(mask, data_format='channels_last', scale=False)
    image = transforms.Resize((image_size, image_size))(image)
    mask = transforms.Resize((mask_size, mask_size))(mask)
    
    # Convert PIL image to Pytorch tensor
    image = transforms.ToTensor()(image)
    mask = transforms.ToTensor()(mask)
    
    image = transforms.Normalize([0.03305855, 0.03305855, 0.01814391], 
                         [0.1787895, 0.12863145, 0.0794853])(image)

    return image, mask

data_transforms = {
    'train': train_transforms,
    'val': valid_transforms
}

image_datasets = {x: create_idrid_loader('{}/{}/'.format(data_dir, x), 
                                         '{}/{}_masks/'.format(data_dir, x), 
                                         transform=data_transforms[x], cache=False)
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                              batch_size=3,
                                              shuffle=True, 
                                              num_workers=8,
                                              pin_memory=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

use_gpu = torch.cuda.is_available()

40 0
40 0
10 0
10 0


# Training the model

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, threshold=0.5):
    since = time.time()
    
    if use_gpu:
        model = model.cuda()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_sn = 0.0
    best_ppv = 0.0
    best_loss = 999999999

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        
        epoch_loss_training = 0.0
        epoch_sensitivity = 0.0
        epoch_positive_predictive_value = 0.0

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            batch_loss_training = 0.0
            batch_sn = 0.0
            batch_ppv = 0.0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data['raw'], data['label']

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())#.type(torch.cuda.LongTensor)
                else:
                    inputs, labels = Variable(inputs), Variable(labels)#.type(torch.LongTensor)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                logits = model(inputs)
                
                loss_for_training = criterion(logits, labels)
                metric_sn  = metrics.SN(logits, labels, threshold) 
                metric_ppv = metrics.PPV(logits, labels, threshold)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss_for_training.backward()
                    optimizer.step()
                    
                # statistics
                batch_loss_training += loss_for_training.data[0] * inputs.size(0)
                batch_sn += metric_sn.data[0] * inputs.size(0)
                batch_ppv += metric_ppv.data[0] * inputs.size(0)

            epoch_loss_training = batch_loss_training / dataset_sizes[phase]
            epoch_sn = batch_sn / dataset_sizes[phase]
            epoch_ppv = batch_ppv / dataset_sizes[phase]
                                   
            print('{} Loss: {:.4f}, SN: {:.4f}, PPV: {:.4f}'.format(phase, epoch_loss_training,
                                                                    epoch_sn, epoch_ppv))
            # deep copy the model
            if (phase == 'val') and (epoch_sn > best_sn): #or (epoch_ppv > best_ppv)):
                best_sn = epoch_sn
                best_ppv = epoch_ppv
                best_loss = epoch_loss_training
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(best_model_wts, join(output_dir, '{}_{}_{}.pth'.format(best_sn, best_ppv, best_loss)))
                
                print('Weights of model saved at {}'.format(join(output_dir, '{}_{}_{}.pth'.format(best_sn, 
                                                                                                   best_ppv, 
                                                                                                   best_loss))))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val SN: {:4f}, Best val PPV: {:4f}, Best Loss: {:4f}'.format(best_sn, best_ppv, best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Config Model

In [5]:
model = RefineNet4Cascade((3, image_size), num_classes=1)

criterion = metrics.criterion_fn

# Observe that all parameters are being optimized
#optimizer = optim.SGD(filter(lambda p:  p.requires_grad, model.parameters()), lr=0.001, momentum=0.9)
optimizer = optim.Adam(filter(lambda p:  p.requires_grad, model.parameters()), lr=0.0001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

# Train and evaluate

In [8]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 1.1694, SN: 0.0000, PPV: nan
val Loss: 1.0816, SN: 0.0000, PPV: nan

Epoch 2/50
----------
train Loss: 1.0889, SN: 0.0000, PPV: nan
val Loss: 1.1080, SN: 0.0000, PPV: nan

Epoch 3/50
----------
train Loss: 1.0752, SN: 0.0000, PPV: nan
val Loss: 1.0529, SN: 0.0000, PPV: nan

Epoch 4/50
----------
train Loss: 1.0138, SN: 0.0000, PPV: nan
val Loss: 0.9910, SN: 0.0000, PPV: nan

Epoch 5/50
----------
train Loss: 0.9530, SN: 0.0119, PPV: nan
val Loss: 0.9591, SN: 0.3938, PPV: 0.2942
Model saved.

Epoch 6/50
----------
train Loss: 0.8866, SN: 0.4064, PPV: 0.3362
val Loss: 0.9624, SN: 0.7367, PPV: 0.1711
Model saved.

Epoch 7/50
----------
train Loss: 0.8087, SN: 0.4722, PPV: 0.3646
val Loss: 0.8272, SN: 0.6512, PPV: 0.2559
Model saved.

Epoch 8/50
----------
train Loss: 0.8160, SN: 0.4916, PPV: 0.3134
val Loss: 0.7794, SN: 0.4854, PPV: 0.3670
Model saved.

Epoch 9/50
----------
train Loss: 0.7599, SN: 0.4715, PPV: nan
val Loss: 0.7506, SN: 0.5036, PPV: 0.403

In [ ]:
val Loss: 0.6283, SN: 0.5830, PPV: 0.5109

In [14]:
weightt_path = join(output_dir, '0.6047431260347367_0.4754905641078949_0.6427530169486999.pth')
model_ft.load_state_dict(torch.load(weightt_path))

In [15]:
optimizer = optim.Adam(filter(lambda p:  p.requires_grad, model.parameters()), lr=0.00001)

In [16]:
model_ft = train_model(model_ft, criterion, optimizer, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.6119, SN: 0.5702, PPV: 0.4913
val Loss: 0.6415, SN: 0.5039, PPV: 0.4245
Model saved.

Epoch 2/50
----------
train Loss: 0.6152, SN: 0.5686, PPV: 0.4731
val Loss: 0.6401, SN: 0.5584, PPV: 0.4814
Model saved.

Epoch 3/50
----------
train Loss: 0.6061, SN: 0.5679, PPV: 0.5090
val Loss: 0.6397, SN: 0.5875, PPV: 0.4489
Model saved.

Epoch 4/50
----------
train Loss: 0.6107, SN: 0.5723, PPV: 0.4784
val Loss: 0.6371, SN: 0.5289, PPV: 0.4455

Epoch 5/50
----------
train Loss: 0.6113, SN: 0.5653, PPV: 0.4778
val Loss: 0.6353, SN: 0.5336, PPV: 0.4397

Epoch 6/50
----------
train Loss: 0.5983, SN: 0.5833, PPV: 0.4837
val Loss: 0.6359, SN: 0.5619, PPV: 0.4655
Model saved.

Epoch 7/50
----------
train Loss: 0.6045, SN: 0.5247, PPV: 0.4663
val Loss: 0.6366, SN: 0.6003, PPV: 0.4443
Model saved.

Epoch 8/50
----------
train Loss: 0.6040, SN: 0.5762, PPV: 0.4611
val Loss: 0.6350, SN: 0.5338, PPV: 0.4541
Model saved.

Epoch 9/50
----------
train Loss: 0.6042, SN: 0.53

In [19]:
model = RefineNet4Cascade((3, image_size), num_classes=1, freeze_resnet=False)
optimizer = optim.Adam(filter(lambda p:  p.requires_grad, model.parameters()), lr=0.0001)
weightt_path = join(output_dir, '0.5830354124307633_0.5109297752380371_0.6283033072948456.pth')
model.load_state_dict(torch.load(weightt_path))

In [20]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.6432, SN: 0.5648, PPV: 0.4868
val Loss: 0.6374, SN: 0.6200, PPV: 0.4946
Weights of model saved at ./weights/0.6199580252170562_0.494629842042923_0.6373553276062012.pth

Epoch 2/50
----------
train Loss: 0.6160, SN: 0.5461, PPV: 0.4653
val Loss: 0.6330, SN: 0.4621, PPV: 0.4363

Epoch 3/50
----------
train Loss: 0.6113, SN: 0.5538, PPV: 0.4614
val Loss: 0.6324, SN: 0.5066, PPV: 0.4503

Epoch 4/50
----------
train Loss: 0.6037, SN: 0.5719, PPV: 0.4780
val Loss: 0.6308, SN: 0.4883, PPV: 0.3858

Epoch 5/50
----------
train Loss: 0.6052, SN: 0.5680, PPV: 0.4607
val Loss: 0.6375, SN: 0.6131, PPV: 0.3824

Epoch 6/50
----------
train Loss: 0.6020, SN: 0.5885, PPV: 0.4979
val Loss: 0.6324, SN: 0.5598, PPV: 0.3695

Epoch 7/50
----------
train Loss: 0.5958, SN: 0.5734, PPV: 0.4877
val Loss: 0.6294, SN: 0.5746, PPV: 0.4863

Epoch 8/50
----------
train Loss: 0.5952, SN: 0.5847, PPV: 0.4972
val Loss: 0.6293, SN: 0.5741, PPV: 0.4704

Epoch 9/50
----------
train Loss

RuntimeError: Unknown error -1

In [6]:
model = RefineNet4Cascade((3, image_size), num_classes=1, freeze_resnet=False)
optimizer = optim.Adam(filter(lambda p:  p.requires_grad, model.parameters()), lr=0.0001)
weightt_path = join(output_dir, '0.6279992282390594_0.46617298722267153_0.6288009941577911.pth')
model.load_state_dict(torch.load(weightt_path))

In [7]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=50)

Epoch 1/50
----------
train Loss: 0.6261, SN: 0.5589, PPV: 0.4849
val Loss: 0.6338, SN: 0.5399, PPV: 0.4405
Weights of model saved at ./weights/0.539866840839386_0.44053341150283815_0.6337691128253937.pth

Epoch 2/50
----------
train Loss: 0.6269, SN: 0.5386, PPV: 0.4749
val Loss: 0.6386, SN: 0.5648, PPV: 0.3804
Weights of model saved at ./weights/0.5648127913475036_0.38043809980154036_0.6386163771152497.pth

Epoch 3/50
----------
train Loss: 0.6165, SN: 0.5395, PPV: 0.4521
val Loss: 0.6362, SN: 0.6521, PPV: 0.4246
Weights of model saved at ./weights/0.6521024376153945_0.42463904321193696_0.6361688792705535.pth

Epoch 4/50
----------
train Loss: 0.6141, SN: 0.5448, PPV: 0.4771
val Loss: 0.6502, SN: 0.6902, PPV: 0.4063
Weights of model saved at ./weights/0.6901501536369323_0.4063425987958908_0.6501660764217376.pth

Epoch 5/50
----------
train Loss: 0.6101, SN: 0.5907, PPV: 0.4657
val Loss: 0.6294, SN: 0.5095, PPV: 0.4016

Epoch 6/50
----------
train Loss: 0.6136, SN: 0.5518, PPV: 0.4971

train Loss: 0.5712, SN: 0.5936, PPV: 0.5071
val Loss: 0.6254, SN: 0.5844, PPV: 0.5156
Weights of model saved at ./weights/0.5843683540821075_0.5155929118394852_0.6254106640815735.pth

Epoch 47/50
----------
train Loss: 0.5665, SN: 0.5834, PPV: 0.4929
val Loss: 0.6472, SN: 0.3947, PPV: 0.5478
Weights of model saved at ./weights/0.39465825706720353_0.5477960288524628_0.6472209334373474.pth

Epoch 48/50
----------
train Loss: 0.5747, SN: 0.5544, PPV: 0.5250
val Loss: 0.6262, SN: 0.5406, PPV: 0.3889
Weights of model saved at ./weights/0.5405613616108894_0.38887912333011626_0.626153975725174.pth

Epoch 49/50
----------
train Loss: 0.5728, SN: 0.5677, PPV: 0.4815
val Loss: 0.6209, SN: 0.5285, PPV: 0.4679
Weights of model saved at ./weights/0.5284724712371827_0.4679022580385208_0.6208513081073761.pth

Epoch 50/50
----------
train Loss: 0.5666, SN: 0.5900, PPV: 0.5176
val Loss: 0.6182, SN: 0.5426, PPV: 0.4932
Weights of model saved at ./weights/0.542623570561409_0.49317572712898256_0.618197792

In [6]:
model = RefineNet4Cascade((3, image_size), num_classes=1, freeze_resnet=False)
optimizer = optim.Adam(filter(lambda p:  p.requires_grad, model.parameters()), lr=0.0001)
weightt_path = join(output_dir, '0.7004039406776428_0.45115841627120973_0.6317032456398011.pth')
model.load_state_dict(torch.load(weightt_path))

In [7]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=100)

Epoch 1/100
----------
train Loss: 0.5909, SN: 0.5667, PPV: 0.4738
val Loss: 0.6284, SN: 0.5942, PPV: 0.4356
Weights of model saved at ./weights/0.5942043602466583_0.43559148609638215_0.6284456193447113.pth

Epoch 2/100
----------
train Loss: 0.5919, SN: 0.5647, PPV: 0.4825
val Loss: 0.6254, SN: 0.6444, PPV: 0.5038
Weights of model saved at ./weights/0.6443693816661835_0.5037580877542496_0.6254413455724717.pth

Epoch 3/100
----------
train Loss: 0.5897, SN: 0.5926, PPV: 0.4948
val Loss: 0.6206, SN: 0.5345, PPV: 0.4637

Epoch 4/100
----------
train Loss: 0.5773, SN: 0.5934, PPV: 0.5070
val Loss: 0.6206, SN: 0.5615, PPV: 0.4550

Epoch 5/100
----------
train Loss: 0.5910, SN: 0.5447, PPV: 0.4733
val Loss: 0.6443, SN: 0.5329, PPV: 0.3458

Epoch 6/100
----------
train Loss: 0.5994, SN: 0.5876, PPV: 0.4493
val Loss: 0.6241, SN: 0.5844, PPV: 0.4359

Epoch 7/100
----------
train Loss: 0.5835, SN: 0.5800, PPV: 0.4927
val Loss: 0.6227, SN: 0.5199, PPV: 0.4835

Epoch 8/100
----------
train Loss: 

val Loss: 0.6308, SN: 0.5953, PPV: 0.5271

Epoch 73/100
----------
train Loss: 0.5785, SN: 0.5771, PPV: 0.4994
val Loss: 0.6276, SN: 0.6680, PPV: 0.4699
Weights of model saved at ./weights/0.6679832637310028_0.46991457641124723_0.6276046454906463.pth

Epoch 74/100
----------
train Loss: 0.5762, SN: 0.5728, PPV: 0.4980
val Loss: 0.6346, SN: 0.5921, PPV: 0.5338

Epoch 75/100
----------
train Loss: 0.5713, SN: 0.5658, PPV: 0.5051
val Loss: 0.6245, SN: 0.5374, PPV: 0.4934

Epoch 76/100
----------
train Loss: 0.5850, SN: 0.5765, PPV: 0.5065
val Loss: 0.6232, SN: 0.6032, PPV: 0.4695

Epoch 77/100
----------
train Loss: 0.5792, SN: 0.5862, PPV: 0.5038
val Loss: 0.6216, SN: 0.6388, PPV: 0.4694

Epoch 78/100
----------
train Loss: 0.5749, SN: 0.5574, PPV: 0.4866
val Loss: 0.6405, SN: 0.4899, PPV: 0.5368

Epoch 79/100
----------
train Loss: 0.5794, SN: 0.5733, PPV: 0.5074
val Loss: 0.6252, SN: 0.6227, PPV: 0.5074

Epoch 80/100
----------
train Loss: 0.5724, SN: 0.5663, PPV: 0.4967
val Loss: 0.62